In [1]:
from src.bayes.BayesianModel import BayesianModel
from src.optimizers.SAMTrainer import SAMTrainer
from src.optimizers.SimpleTrainer import SimpleTrainer
%load_ext autoreload
%autoreload 2
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
import torch
from src.optimizers.SimpleTrainer import SimpleTrainer
from src.utils.evaluation import evaluate_run
from src.utils import dataset
from matplotlib import pyplot as plt
from src.optimizers import interval_narrowers
from src.optimizers.ConstrainedVolumeMultiphaseTrainer import ConstrainedVolumeMultiphaseTrainer
from src.optimizers.PGDTrainer import PGDTrainer
from src.optimizers.HypercubeTrainer import HypercubeTrainer, ConstrainedVolumeTrainer
from src.utils.evaluation import evaluate_accuracy, evaluate_fgsm_accuracy, evaluate_certified_adv_accuracy
from src.cert import Safebox
from src.bayes.DiagonalGaussianBayesianModel import DiagonalGaussianBayesianModel
from src.bayes.UniformBayesianModel import UniformBayesianModel
from src.bayes.BayesianModel import BayesianModel
import math
from src.optimizers.SAMTrainer import SAMTrainer
from torch.utils.data import DataLoader, Dataset

In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
mnist_train_test, mnist_val = dataset.get_mnist_dataset(root="./data")
mnist_train, mnist_test = dataset.split_dataset(mnist_train_test, split_proportion=0.8)
print("Train Dataset Size: ", len(mnist_train))
print("Test Dataset Size: ", len(mnist_test))
print("Validation Dataset Size: ", len(mnist_val))

Train Dataset Size:  48000
Test Dataset Size:  12000
Validation Dataset Size:  10000


In [100]:
fashion_mnist_train_test, fashion_mnist_val = dataset.get_fashion_mnist_dataset(root="./data")
fashion_mnist_train, fashion_mnist_test = dataset.split_dataset(fashion_mnist_train_test, split_proportion=0.8)
print("Train Dataset Size: ", len(fashion_mnist_train))
print("Test Dataset Size: ", len(fashion_mnist_test))
print("Validation Dataset Size: ", len(fashion_mnist_val))


Train Dataset Size:  48000
Test Dataset Size:  12000
Validation Dataset Size:  10000


In [5]:
def get_model(output_dim=10):
    """Returns a simple CNN model."""
    model = torch.nn.Sequential(
        torch.nn.Conv2d(1, 8, kernel_size=5, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.Conv2d(8, 1, kernel_size=5, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.Flatten(),
        torch.nn.Linear(576, output_dim),
    ).to(DEVICE)
    return model

def make_bayesian_gaussian(interval_model: torch.nn.Sequential, bound_mul: float=70) -> BayesianModel:
    """
    This function assumes bound model to be already centered so it will only use W_u.
    """
    center_model = Safebox.bmodelToModel(interval_model)
    std_params = []
    for layerInterval in interval_model:
        if isinstance(layerInterval, Safebox.BDense) or isinstance(layerInterval, Safebox.BConv2d):
            std_params.append((layerInterval.W_u*bound_mul).clone().detach())
            std_params.append((layerInterval.b_u*bound_mul).clone().detach())
    return DiagonalGaussianBayesianModel(
        center_model, std_params
    )
def make_bayesian_uniform(interval_model: torch.nn.Sequential, bound_mul: float=70) -> BayesianModel:
    bound_params = []
    center_model = Safebox.bmodelToModel(interval_model)
    for layerInterval in interval_model:
        if isinstance(layerInterval, Safebox.BDense) or isinstance(layerInterval, Safebox.BConv2d):
            bound_params.append((layerInterval.W_u*100).clone().detach())
            bound_params.append((layerInterval.b_u*100).clone().detach())
        

    return UniformBayesianModel(
        center_model, bound_params
    )

In [6]:
# Setting random seeds for reproducibility
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

h_model = get_model(output_dim=10)
optimizer = HypercubeTrainer(h_model, device=DEVICE)
multiphase_trainer = ConstrainedVolumeMultiphaseTrainer(
    optimizer, 
    inflate_function=interval_narrowers.inflate_multiplier(2.0),
    narrow_function=interval_narrowers.narrower_halver(0.5),
    starting_value=1e-4*0.5, 
    min_acc_limit=0.78,
    quiet=False
)
restart = not multiphase_trainer.train(
    7, mnist_val,
    mnist_train, mnist_val, loss_obj=0.0, max_iters=100, batch_size=64, lr=1e-3
)

========== Started Multiphase Trainer for 7 phases ==========
First time train is called. Initial training phase started with volume 5e-05.


100%|██████████| 100/100 [00:09<00:00, 10.81it/s, loss=0.666, min_val_acc=0.835]


=> Initial center accuracy is 0.8593000173568726
-> Starting phase 6
-> Current Volume interval : [5e-05, INFINITY]


100%|██████████| 100/100 [00:07<00:00, 13.87it/s, loss=0.446, min_val_acc=0.876]


-> Phase done
-> Center Accuracy is 0.8878999948501587.
-> Generalization is above minimum accuracy by 0.10789999485015866.!
-> Starting phase 6
-> Current Volume interval : [0.0001, INFINITY]


...

-> Phase done
-> Center Accuracy is 0.9143999814987183.
-> Generalization is above minimum accuracy by 0.13439998149871824.!
-> Starting phase 6
-> Current Volume interval : [0.0008, INFINITY]


100%|██████████| 100/100 [00:07<00:00, 14.00it/s, loss=0.461, min_val_acc=0.754]


-> Phase done
-> Center Accuracy is 0.9146000146865845.
-> Generalization is above minimum accuracy by 0.13460001468658445.!
-> Starting phase 6
-> Current Volume interval : [0.0016, INFINITY]


100%|██████████| 100/100 [00:07<00:00, 14.24it/s, loss=1.64, min_val_acc=0.572]


-> Phase done
-> Center Accuracy is 0.90420001745224.
-> Generalization is above minimum accuracy by 0.12420001745223996.!
Training succeeded !


In [120]:
X, y = next(iter(DataLoader(mnist_test, batch_size=200, shuffle=True)))
X_noise, _ = next(iter(DataLoader(fashion_mnist_test, batch_size=200, shuffle=True)))
X_noise_detection = torch.concat([X_noise, X], dim=0)
y_noise_detection = torch.concat([torch.ones_like(y), torch.zeros_like(y)])



In [121]:

interval_model = multiphase_trainer.result()
bayesian_model_gaussian = make_bayesian_gaussian(interval_model, bound_mul=0.0)
mean, std, entropy = bayesian_model_gaussian.predict(X, n_samples=100, device=DEVICE)
mean, std, entropy = mean.cpu(), std.cpu(), entropy.cpu()
print("Accuracy Bayesian neural network ", (mean.argmax(dim=1)==y).sum()/200)
mean, std, entropy = bayesian_model_gaussian.predict(X_noise_detection, n_samples=100, device=DEVICE)
mean, std, entropy = mean.cpu(), std.cpu(), entropy.cpu()
print("Accuracy in noise detection ", ((entropy>1.0) == y_noise_detection.cpu()).sum()/400)

Accuracy Bayesian neural network  tensor(0.8700)
Accuracy in noise detection  tensor(0.7075)


In [98]:
adam_model = get_model()        
trainer = SimpleTrainer(adam_model, device=DEVICE, quiet=False, acc_evaluation_steps=70)
trainer.train(mnist_train, mnist_val, loss_obj=0.0, max_iters=700, batch_size=64, lr=1e-3)
adam_model = trainer.result()


100%|██████████| 700/700 [00:48<00:00, 14.50it/s, loss=0.178, val_acc=0.93]  


In [122]:
interval_model = Safebox.modelToBModel(adam_model)
Safebox.assign_epsilon(interval_model, 3*1e-3)
bayesian_model_gaussian = make_bayesian_gaussian(interval_model, bound_mul=0.0)
mean, std, entropy = bayesian_model_gaussian.predict(X, n_samples=100, device=DEVICE)
mean, std, entropy = mean.cpu(), std.cpu(), entropy.cpu()
print("Accuracy Bayesian neural network ", (mean.argmax(dim=1)==y).sum()/200)
mean, std, entropy = bayesian_model_gaussian.predict(X_noise_detection, n_samples=100, device=DEVICE)
mean, std, entropy = mean.cpu(), std.cpu(), entropy.cpu()
print("Accuracy in noise detection ", ((entropy>1.0) == y_noise_detection.cpu()).sum()/400)

Accuracy Bayesian neural network  tensor(0.9200)
Accuracy in noise detection  tensor(0.6775)
